In [45]:
'''
- Purpose of this code is to return a compiled view of financial statements of any copy to be able to manipulate them individually
- Returns a dataframe object
'''
import pandas as pd
from pandas import DataFrame as df
import csv
import os
import sqlite3
import numpy as np
import string
os.chdir('F:\Database Project\SQLLITE Studio Files')
con = sqlite3.connect('financial_database.db3')
con.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')

In [46]:
#input the company name you're looking for, ex: Coca Cola (all caps)
s='COSTCO'

In [47]:
#retrieve all the 10-K forms from sub concerning the company s
comp_tab = pd.read_sql_query('''
select * from sub
where 
    name LIKE '{comp}%'
    AND form = '10-K'
    AND fp = 'FY'
order by fy
limit 1000;
'''.format(comp = s), con) 

In [48]:
comp_tab['fy']

0    2012.0
1    2013.0
2    2014.0
3    2015.0
4    2016.0
Name: fy, dtype: float64

In [49]:
#if that query contains the right company, move forward with the adsh's returned
#make adsh's into a string to be read in sqlite code
adsh_list = string.replace(str(comp_tab['adsh'].tolist()),'u','')
adsh_list = string.replace(adsh_list,'[','')
adsh_list = string.replace(adsh_list,']','')

#find the dates the reports were published
#make the dates into a string to be read in sqlite code
ddate_list = string.replace(str(comp_tab['period'].tolist()), 'L', '')
ddate_list = string.replace(ddate_list,'[','')
ddate_list = string.replace(ddate_list,']','')

In [50]:
ren_tab = pd.read_sql_query('''
SELECT
    *
FROM ren   
WHERE 
    adsh in ({adsh})
    and longname like '%Statements of Income%'
order by adsh, report, longname
LIMIT 5000;'''.format(adsh = adsh_list, ddate = ddate_list), con)

In [51]:
ren_tab['adsh']

0    0000909832-14-000021
1    0000909832-15-000014
2    0000909832-16-000032
3    0001193125-12-428890
4    0001445305-13-002422
Name: adsh, dtype: object

In [52]:
#make the first adsh read the model for the other ones
first_tab = pd.read_sql_query('''
SELECT
    adsh, report, line, stmt, tag, ddate, qtrs, value, negating
FROM pre   
 JOIN num using ( adsh, tag )
WHERE 
    num.adsh in ({adsh})
    AND stmt in ('IS')
    /*shows only values which are totalled from other values*/
    /*AND prole = 'totalLabel'*/
    AND ddate in ({ddate})
    AND qtrs in (0, 4)
    AND dimh = '0x00000000'
order by adsh, report, line
LIMIT 5000;'''.format(adsh = adsh_list, ddate = ddate_list), con)

In [53]:
first_tab

,adsh,report,line,stmt,tag,ddate,qtrs,value,negating
0,0000909832-14-000021,4,2,IS,SalesRevenueNet,20120831,4,9.706200e+10,0
1,0000909832-14-000021,4,2,IS,SalesRevenueNet,20130831,4,1.028700e+11,0
2,0000909832-14-000021,4,2,IS,SalesRevenueNet,20140831,4,1.102120e+11,0
3,0000909832-14-000021,4,3,IS,RevenueFromEnrollmentAndRegistrationFeesExclud...,20140831,4,2.428000e+09,0
4,0000909832-14-000021,4,3,IS,RevenueFromEnrollmentAndRegistrationFeesExclud...,20120831,4,2.075000e+09,0
5,0000909832-14-000021,4,3,IS,RevenueFromEnrollmentAndRegistrationFeesExclud...,20130831,4,2.286000e+09,0
6,0000909832-14-000021,4,4,IS,Revenues,20130831,4,1.051560e+11,0
7,0000909832-14-000021,4,4,IS,Revenues,20120831,4,9.913700e+10,0
8,0000909832-14-000021,4,4,IS,Revenues,20140831,4,1.126400e+11,0
9,0000909832-14-000021,4,6,IS,CostOfGoodsSold,20120831,4,8.682300e+10,0


In [54]:
#drop the duplicate values which are found on more than one statement
first_tab = first_tab.drop_duplicates(subset = ['stmt','tag','ddate','value']).sort_values(['adsh','report','line'])

In [55]:
first_tab

,adsh,report,line,stmt,tag,ddate,qtrs,value,negating
0,0000909832-14-000021,4,2,IS,SalesRevenueNet,20120831,4,9.706200e+10,0
1,0000909832-14-000021,4,2,IS,SalesRevenueNet,20130831,4,1.028700e+11,0
2,0000909832-14-000021,4,2,IS,SalesRevenueNet,20140831,4,1.102120e+11,0
3,0000909832-14-000021,4,3,IS,RevenueFromEnrollmentAndRegistrationFeesExclud...,20140831,4,2.428000e+09,0
4,0000909832-14-000021,4,3,IS,RevenueFromEnrollmentAndRegistrationFeesExclud...,20120831,4,2.075000e+09,0
5,0000909832-14-000021,4,3,IS,RevenueFromEnrollmentAndRegistrationFeesExclud...,20130831,4,2.286000e+09,0
6,0000909832-14-000021,4,4,IS,Revenues,20130831,4,1.051560e+11,0
7,0000909832-14-000021,4,4,IS,Revenues,20120831,4,9.913700e+10,0
8,0000909832-14-000021,4,4,IS,Revenues,20140831,4,1.126400e+11,0
9,0000909832-14-000021,4,6,IS,CostOfGoodsSold,20120831,4,8.682300e+10,0


In [56]:
#Make the index a combination of the statement and tag to be matched on the index
sec_tab = first_tab[['report','line','stmt','tag']].drop_duplicates(subset = ['stmt','tag'], keep='first')
sec_tab.index = sec_tab['stmt']+' - ' + sec_tab['tag']

In [57]:
tab_list = []
#the first table is the sec_tab
tab_list.append(sec_tab)
for i in range(len(comp_tab['period'])):
    date = comp_tab['period'][i]
    small_tab = first_tab.loc[first_tab['ddate'] == date].sort_values(['report','line'])
    #make the index the same as the sec tab
    small_tab.index = small_tab['stmt']+' - ' + small_tab['tag']
    #keep only the values
    small_tab = small_tab['value']
    small_tab = small_tab.rename(columns = {'value':date})
    #add the table to a list of tables
    tab_list.append(small_tab)

In [58]:
#concatenate all the above dataframes
result = pd.concat(tab_list, axis = 1)

In [59]:
#rename columns with dates
for i in range(len(comp_tab['period'])):
    date = comp_tab['period'][i]
    result = result.rename(columns = {i:date})
result = result.sort_values(['report','line'])
#reindex with numbers
result.index = np.arange(len(result))

In [37]:
result.to_csv('costis.csv')

In [62]:
result

,report,line,stmt,tag,20120831,20130831,20140831,20150831,20160831
0,4,2,IS,SalesRevenueNet,9.706200e+10,1.028700e+11,1.102120e+11,1.136660e+11,1.160730e+11
1,4,3,IS,RevenueFromEnrollmentAndRegistrationFeesExclud...,2.075000e+09,2.286000e+09,2.428000e+09,2.533000e+09,2.646000e+09
2,4,4,IS,Revenues,9.913700e+10,1.051560e+11,1.126400e+11,1.161990e+11,1.187190e+11
3,4,6,IS,CostOfGoodsSold,8.682300e+10,9.194800e+10,9.845800e+10,1.010650e+11,1.029010e+11
4,4,7,IS,SellingGeneralAndAdministrativeExpense,9.518000e+09,1.010400e+10,1.089900e+10,1.144500e+10,1.206800e+10
5,4,8,IS,PreOpeningCosts,3.700000e+07,5.100000e+07,6.300000e+07,6.500000e+07,7.800000e+07
6,4,9,IS,OperatingIncomeLoss,2.759000e+09,3.053000e+09,3.220000e+09,3.624000e+09,3.672000e+09
7,4,11,IS,InterestExpense,9.500000e+07,9.900000e+07,1.130000e+08,1.240000e+08,1.330000e+08
8,4,12,IS,InterestAndOtherIncome,1.030000e+08,9.700000e+07,9.000000e+07,1.040000e+08,8.000000e+07
9,4,13,IS,IncomeLossFromContinuingOperationsBeforeIncome...,2.767000e+09,3.051000e+09,3.197000e+09,3.604000e+09,3.619000e+09
